Optional check of timing after each cell

In [1]:
%load_ext autotime

Necessary imports

In [2]:
import os
import gzip
from datetime import datetime
import numpy as np
import pandas as pd
import cudf
from pandas import Series, DataFrame
import sklearn.utils

time: 448 ms


Julian McAuley's code to unzip the gzip file and load it into a pandas dataframe.

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

time: 883 µs


Set up the DATA_DIR environment variable to point to the review data. Used in all the code.

In [4]:
data_dir = os.environ['DATA_DIR'] + '/'

time: 3.9 ms


Read the full gzipped data into a data frame.

In [5]:
jdf = getDF(data_dir + 'reviews_Books_5.json.gz') # 'json' df

time: 6min 39s


In [6]:
jdf

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5.0,Wonderful!,1355616000,"12 16, 2012"
1,A2S166WSCFIFP5,000100039X,"adead_poet@hotmail.com ""adead_poet@hotmail.com""","[0, 2]",This is one my must have books. It is a master...,5.0,close to god,1071100800,"12 11, 2003"
2,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""","[0, 0]",This book provides a reflection that you can a...,5.0,Must Read for Life Afficianados,1390003200,"01 18, 2014"
3,A1MOSTXNIO5MPJ,000100039X,Alan Krug,"[0, 0]",I first read THE PROPHET in college back in th...,5.0,Timeless for every good and bad time in your l...,1317081600,"09 27, 2011"
4,A2XQ5LZHTD4AFT,000100039X,Alaturka,"[7, 9]",A timeless classic. It is a very demanding an...,5.0,A Modern Rumi,1033948800,"10 7, 2002"
5,A3V1MKC2BVWY48,000100039X,Alex Dawson,"[0, 0]",Reading this made my mind feel like a still po...,5.0,This book will bring you peace,1390780800,"01 27, 2014"
6,A12387207U8U24,000100039X,Alex,"[0, 0]","As you read, Gibran's poetry brings spiritual ...",5.0,Graet Work,1206662400,"03 28, 2008"
7,A29TRDMK51GKZR,000100039X,Alpine Plume,"[0, 0]","Deep, moving dramatic verses of the heart and ...",5.0,Such Beauty,1383436800,"11 3, 2013"
8,A3FI0744PG1WYG,000100039X,"Always Reading ""tkm""","[0, 0]",This is a timeless classic. Over the years I'...,5.0,The Prophet,1390953600,"01 29, 2014"
9,A2LBBQHYLEHM7P,000100039X,"Amazon Customer ""Full Frontal Nerdity""","[0, 0]",An amazing work. Realizing extensive use of Bi...,5.0,A Modern Classic,1379808000,"09 22, 2013"


time: 35.8 ms


We're keeping asin (amazon id), reviewText, and overall (the rating, 1.0 to 5.0).
There are enough reviews that we can take the smallest number of reviews, that for one star, and use that number
of the rest of the reviews to get a balance.

In [7]:
item_counts = jdf['overall'].value_counts()
item_counts

5.0    4980815
4.0    2223094
3.0     955189
2.0     415110
1.0     323833
Name: overall, dtype: int64

time: 78.5 ms


In [8]:
minimum_review_counts = item_counts[1]
minimum_review_counts

323833

time: 1.61 ms


Loading from gzip takes a long time and includes columns we don't need.
Save the relevant columns as a csv, to load and have around for later analysis.

In [9]:
jdf.to_csv(data_dir + 'reviews_1_to_5.csv', index=None, columns=['asin', 'reviewText', 'overall'])

time: 1min 42s


We no longer need the dataframe from the json file.

In [10]:
del jdf

time: 299 µs


Load the trimmed dataframe.

In [11]:
df = pd.read_csv(data_dir + 'reviews_1_to_5.csv')

time: 51.7 s


Shuffle it.

In [12]:
df = sklearn.utils.shuffle(df)
df = df.reset_index(drop=True)

time: 5.62 s


Use the same number of reviews from each category, which means the same number as the category with the least reviews.

In [13]:
df1 = df[df['overall'] == 1.0]
df1 = df1.reset_index(drop=True)
df2 = df[df['overall'] == 2.0]
df2 = df2.reset_index(drop=True)
df3 = df[df['overall'] == 3.0]
df3 = df3.reset_index(drop=True)
df4 = df[df['overall'] == 4.0]
df4 = df4.reset_index(drop=True)
df5 = df[df['overall'] == 5.0]
df5 = df5.reset_index(drop=True)

time: 2.39 s


In [14]:
df2 = df2.truncate(before=0, after=minimum_review_counts - 1)
df3 = df3.truncate(before=0, after=minimum_review_counts - 1)
df4 = df4.truncate(before=0, after=minimum_review_counts - 1)
df5 = df5.truncate(before=0, after=minimum_review_counts - 1)

time: 606 ms


Make a new dataframe from the right number of 1 to 5 reviews

In [15]:
dfconcat = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

time: 217 ms


Dataframe is currently all 1, 2, 3, 4, 5, so shuffle again

In [16]:
dfconcat = sklearn.utils.shuffle(dfconcat)
dfconcat = dfconcat.reset_index(drop=True)

time: 983 ms


It's ready to use, so write out the file.

In [18]:
dfconcat.to_csv(data_dir + 'reviews_1_to_5_shuffle.csv', index=None)

time: 18.8 s
